In [1]:
import pylangacq as pla

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
import glob

In [5]:
import os

In [6]:
pla.__version__

'0.11.0'

In [7]:
# data = pla.read_chat('./Pitt/*.cha')

Data grab

In [8]:
cookie_data = pla.read_chat('./Pitt/*/cookie/*.cha')

In [9]:
sentence_data = pla.read_chat('./Pitt/*/sentence/*.cha')

In [10]:
sentence_fnames = glob.glob(os.getcwd()+'/Pitt/*/sentence/*.cha')

In [14]:
sentence_fnames = sorted(sentence_fnames)

In [15]:
sentence_fnames[0]

'/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/001-0.cha'

In [16]:
# Control, Dementia
cookie_fnames = glob.glob(os.getcwd()+'/Pitt/*/cookie/*.cha')

In [17]:
cookie_fnames = sorted(cookie_fnames)

In [89]:
len(sentence_fnames+cookie_fnames)

792

In [90]:
# for random batching/data grabbing 
fnames_dict = {i:fname for (i,fname) in enumerate(sentence_fnames+cookie_fnames)}

Data exploration

In [18]:
len(cookie_data)

552

In [19]:
len(sentence_data)

240

In [20]:
cookie_data.number_of_utterances('PAR')

7054

Targets (MMSE scores)

In [21]:
# fill missing mmse with group's mean mmse
def get_targets(dataset,filenames):
    
    targets = []
    groups = {}
    
#     for fname in sorted(filenames):
    for fname in filenames:
        # PAR MMSE
#         print(fname)
        target = dataset.headers()[fname]['Participants']['PAR']['education']
        group = dataset.headers()[fname]['Participants']['PAR']['group']
        
        # to be replaced with group mean
        if target is '':
            targets.append(group)
            
        else:
            targets.append(int(target))
            if group in groups.keys():
                groups[group].append(int(target))
            else:
                groups[group] = [int(target)]
                
#     print('targets',targets)            
#     print('groups',groups)
    
    for g in groups.keys():
        m = np.mean(groups[g])
        groups[g] = int(m) if m % 1 > 5 else int(m) + 1
    
    #TODO replace with tensor
    targetseries = pd.Series(targets)
    
#     print('groups_mean',groups)
    
    for g in groups.keys():
        targetseries[[i for i,x in enumerate(targets) if x==g]] = groups[g]

    return targetseries

In [22]:
sentence_data_targets = get_targets(sentence_data,sentence_fnames)

In [23]:
cookie_data_targets = get_targets(cookie_data,cookie_fnames)

In [19]:
len(sentence_data_targets) == len(sentence_data)

True

In [20]:
len(cookie_data_targets) == len(cookie_data)

True

BUILD VOCAB

In [71]:
vocabset = set()

In [63]:
len(cookie_fnames)

552

In [72]:
for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data)]: 
    for i in range(len(corpus)):
        tokens = set([tokenraw for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
        vocabset.update(tokens)

In [74]:
len(vocabset)

2188

In [ ]:
for i in range(len(sentence_fnames)):

BUILD MOR: POS + GRAMMATICAL CATEGORIES

In [75]:
posset = set()

In [76]:
for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data)]: 
    for i in range(len(corpus)):
        pos = set([pos for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
        posset.update(pos)

In [82]:
len(posset)

71

In [87]:
# to concat to embedding tensor 
pos_dict = {i:pos for (i,pos) in enumerate(posset)}

CLEAN POS, VOCAB


note '' pos, '.' token: <EOS> 

In [93]:
#https://github.com/Blosc/bcolz
import bcolz